In [ ]:
import os
from dotenv import load_dotenv
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination    
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_agentchat.messages import TextMessage
from openai import OpenAI

from litellm import completion

In [2]:
load_dotenv()

True

In [3]:
mails = [
    #TO-DO: Add category-0 and category-1 email samples
    {
        "birthday":
     [
         {
             "subject": "Wishing You a Very Happy Birthday",
          "body": '''Dear [Recipients Name],
          On behalf of the team, I would like to extend my warmest wishes to you on your birthday. May this year bring you continued success, good health, and happiness.
          We truly appreciate your guidance and support, and we look forward to achieving many more milestones together.
          Wishing you a wonderful day and a prosperous year ahead.'''
          },
         
         {
             "subject": "Happy Birthday",
          "body": '''Wishing you a very Happy Birthday! May your special day be filled with joy and may the year ahead bring you both personal happiness and professional success.
          Its a pleasure working with you, and I hope you take some time today to celebrate and enjoy the occasion.'''
          },
          
          {
              "subject": "Birthday Wishes",
           "body": '''Many happy returns of the day! I hope this birthday marks the beginning of another successful and fulfilling year for you.
           Your dedication and leadership inspire those around you, and I wish you continued achievements along with moments of happiness and relaxation.
           Enjoy your day to the fullest.'''
           }
           ]
           },
           {
               "actionables":
               [
                   {
                       "subject": "Request for Attendance Regularization Approval",
                       "body": '''I hope you are doing well.
                       I would like to request your approval for my attendance regularization for Week 3 (27th October to 2nd November 2025). I have already regularized Week 4 (3rd November to 9th November 2025).
                       Kindly review and approve the request at your earliest convenience so that my records remain updated. I assure you that I will ensure timely regularization going forward.
                       Thank you for your support.'''
                       },
                       
                       {
                           "subject": "Follow-Up on Pending Timesheet Approval",
                           "body": '''I am writing to follow up on my earlier request regarding attendance regularization for Week 3 (27th October to 2nd November 2025). As this impacts payroll and compliance, I kindly request you to review and approve the request at the earliest.
                           If any additional information or clarification is required from my side, please let me know and I will provide it immediately.
                           Thank you for your attention to this matter.'''
                           },
                           
                           {
                               "subject": '''Attendance Regularization to Request for Approval''',
                               "body": '''As discussed, I missed regularizing my timesheet for Week 3 (27th October to 2nd November 2025), though I have already completed Week 4 (3rd November to 9th November 2025).
                               For visibility and timely approval, I am looping in Name and Name on this thread. I kindly request your collective review and approval so that my attendance records remain accurate.
                               I apologize for the oversight and assure you that I will adhere to the timelines henceforth.
                               Thank you for your understanding and support.'''
                               }
                               ]
                               },
                               {
                                   "meetings":
                                   [
                                       {
                                           "subject": "Request to Schedule a Meeting on [Topic]",
                                           "body": '''I hope you are doing well.I would like to request a meeting to discuss [specific topic, e.g., project updates, performance goals, or onboarding progress]. Please let me know your availability this week so that we can finalize a suitable time.
                                           The discussion will help align expectations and ensure smooth progress on the tasks at hand.
                                           Looking forward to your confirmation.'''
                                           },
                                           
                                           {
                                               "subject": "Agenda for Upcoming Meeting on [Date]",
                                               "body": '''Thank you for confirming the meeting scheduled on [date and time]. To ensure an effective discussion, I am sharing the proposed agenda:
                                               Review of current progress
                                               Key challenges and solutions
                                               Next steps and responsibilities
                                               Please let me know if you would like to add or adjust any points. This will help us make the best use of our time together.
                                               Looking forward to a productive discussion.'''
                                               },
                                               
                                               {
                                                   "subject": "Thank You for Todays Meeting",
                                                   "body": '''Thank you for taking the time to meet today. I found our discussion on [topic] very insightful and valuable.
                                                   As agreed, the key action items are:
                                                   [Action Item 1]: Assigned to [Name]
                                                   [Action Item 2]: Assigned to [Name]
                                                   [Action Item 3]: Timeline [Date]
                                                   I will ensure that these points are tracked and followed up accordingly. Please let me know if I missed anything or if further clarification is needed.'''
                                                   }
                                                   ]
                                                   }
                                                   ]

In [4]:
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY")
API_VERSION = "2024-05-01-preview"
AZURE_DEPLOYMENT_NAME = os.getenv("DEPLOYMENT_NAME")

In [17]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_email_sample",
            "description": "Get a sample email based on the category provided.",
            "parameters": {
                "type": "object",
                "properties": {
                    "category": {
                        "type": "string",
                        "description": "The category of the email (e.g., birthday, actionables, meetings)."
                    }
                },
                "required": ["category"]
            },
        }
    }
]

messages = [{ "role": "user", "content": "Generate a email for birthday wish" }]

In [ ]:
response = completion(
    model="ollama/llama3.1",
    messages=messages,
    tools=tools
)

In [ ]:
model_client = AzureOpenAIChatCompletionClient(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_KEY,
    api_version=API_VERSION,
    model=AZURE_DEPLOYMENT_NAME
)   


In [ ]:
agent = AssistantAgent(
    name="EmailAssistant",
    description="An assistant that helps to draft customized emails based on different occasions.",
    model_client=model_client,
    system_message = '''You are an expert email assistant. Your task is to help draft customized emails based on different occasions such as birthdays, meetings, and action items. Use the provided email templates to create personalized messages that are professional and engaging.'''
)

In [7]:
birthday_agent = AssistantAgent(
    "BirthdayEmailAssistant",
    description="An assistant that specializes in drafting customized birthday emails.",
    model_client=model_client,
    system_message = '''You are an expert birthday email assistant. Your task is to help draft customized birthday emails using the provided email templates. Ensure that the messages are warm, professional, and tailored to the recipient's preferences.'''
)

In [ ]:
actionable_agent = AssistantAgent(
    "ActionableEmailAssistant", 
    description="An assistant that specializes in drafting actionable emails.",
    model_client=model_client,
    system_message = '''You are an expert actionable email assistant. Your task is to help draft emails that include clear calls to action and are tailored to the recipient's needs.'''
)

In [ ]:
meeting_agent = AssistantAgent(
    "MeetingEmailAssistant",
    description="An assistant that specializes in drafting meeting-related emails.",
    model_client=model_client,
    system_message = '''You are an expert meeting email assistant. Your task is to help draft emails that schedule, reschedule, or follow up on meetings.'''
)


In [ ]:
comparison_agent = AssistantAgent(
    "ComparisonEmailAssistant",
    description="An assistant that specializes in comparing emails.",
    model_client=model_client,
    system_message="You are an expert comparison email assistant. Your task is to segregate the user mail into categories mentioned in the {mails}."
)

In [ ]:
text_mention_termination = TextMentionTermination("TERMINATE")
max_message_termination = MaxMessageTermination(max_messages = 10)
termination = text_mention_termination | max_message_termination

In [ ]:
team = SelectorGroupChat(
    [birthday_agent, actionable_agent],
    model_client=model_client,
    termination_condition=termination
)

In [18]:
# await Console(team.run_stream())
# response = await model_client.create(
    # model="yueqis/non_web_mcp-qwen-coder-7b-1epoch-30k-5e-5",
    # prompt = f'''Segregate the following emails into categories: 
    # 1. Birthday Emails
    # 2. Actionable Emails
    # 3. Meeting Emails

    # Emails: {mails}

    # Respond with the category name followed by the email details.'''
    # messages = "Hello!"
    #     }
    # ]
# )


response = await birthday_agent.on_messages([TextMessage(content = '''Subject: Happy Birthday John! 
                                                        You are a powerhouse and on this special day you deserve every health, happiness, prosperity, and everything you wish for. 
As you celebrate another year in your life, we wish you all that and much more. 
May You Have A Wonderful Year Ahead
 ''', source = "user")], cancellation_token= None)

d:\Capstone_Project\.capstone\Lib\site-packages\autogen_agentchat\agents\_assistant_agent.py:1109: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  model_result = await model_client.create(


In [19]:
response.chat_message.content

'Subject: Wishing You a Happy Birthday, John!  \n\nDear John,  \n\nHappy Birthday to an incredible leader and an absolute powerhouse! On this special day, you truly deserve every ounce of health, happiness, prosperity, and all the wonderful things life has to offer.  \n\nAs you celebrate another milestone in your journey, I hope it’s filled with everything you enjoy most – maybe a round of golf to relax and the chance to dive into a new sci-fi adventure.  \n\nHere’s to an amazing year ahead, full of success and joy. Thank you for being such an inspiring manager and guiding all of us with your expertise and kindness.  \n\nHave a fantastic birthday, John!  \n\nBest regards,  \n[Your Name]  '

SyntaxError: invalid syntax (3807844594.py, line 1)